# Optimization

## Import Libraries

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd

from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.core.problem import ElementwiseProblem

from src import config
import src.optimization_utils as ou
import src.solutions as s

## Load Models

In [ ]:
# Mechanical failure model
mech_fail_bundle = ou.load_model_bundle(config.MECH_FAIL_MODEL)

# Delta models from notebook 03
L4S1_bundle = ou.load_model_bundle(config.L4S1_MODEL)
LL_bundle = ou.load_model_bundle(config.LL_MODEL)
T4PA_bundle = ou.load_model_bundle(config.T4PA_MODEL)
L1PA_bundle = ou.load_model_bundle(config.L1PA_MODEL)

# Delta models from notebook 04
SVA_bundle = ou.load_model_bundle(config.SVA_MODEL)
SS_bundle = ou.load_model_bundle(config.SS_MODEL)
GT_bundle = ou.load_model_bundle(config.GLOBAL_TILT_MODEL)

print("Loaded models:")
print(f"  - Mechanical failure: {mech_fail_bundle.get('model_name', 'N/A')}")
print(f"  - L4S1: {L4S1_bundle.get('model_name', 'N/A')}")
print(f"  - LL: {LL_bundle.get('model_name', 'N/A')}")
print(f"  - T4PA: {T4PA_bundle.get('model_name', 'N/A')}")
print(f"  - L1PA: {L1PA_bundle.get('model_name', 'N/A')}")
print(f"  - SVA: {SVA_bundle.get('model_name', 'N/A')}")
print(f"  - SS: {SS_bundle.get('model_name', 'N/A')}")
print(f"  - Global Tilt: {GT_bundle.get('model_name', 'N/A')}")

In [ ]:
UIV_CHOICES, xl, xu = ou.get_decision_config()

In [ ]:
print("UIV_CHOICES:", UIV_CHOICES)
print("xl:", xl)
print("xu:", xu)

cols = ["uiv_code","num_fused_levels","ALIF","XLIF","TLIF","num_rods","num_screws","osteotomy"]
print(pd.DataFrame([xl, xu], index=["xl","xu"], columns=cols))

## Test Patient w fixed parameters

In [ ]:
patient_fixed = {
    "age": 65,
    "sex": "FEMALE",
    "bmi": 18.48,
    "C7CSVL_preop": -6.1,
    "SVA_preop": 40.7,
    "T4PA_preop": 7.4,
    "L1PA_preop": 9.1,
    "LL_preop": 51.8,
    "L4S1_preop": 32.5,
    "PT_preop": 14.1,
    "PI_preop": 45.8,
    "SS_preop": 31.7,
    "cobb_main_curve_preop": 60.2,
    "FC_preop": 10.6,
    "tscore_femneck_preop": -0.5,
    "HU_UIV_preop": 229,
    "HU_UIVplus1_preop": 245,
    "HU_UIVplus2_preop": 248,
}

ou.debug_candidate(
    x=np.array([0, 10, 1, 0, 0, 2, 20, 0]),
    patient_fixed=patient_fixed,
    bundle=mech_fail_bundle,
    uiv_choices=UIV_CHOICES,
)

## Build optimization problem

In [ ]:
def make_problem():
    def _evaluate(self, x, out, *args, **kwargs):
        f = ou.fitness_mech_fail_only(x, patient_fixed, mech_fail_bundle, UIV_CHOICES)
        out["F"] = np.array([f], dtype=float)

    ProblemType = type(
        "SpineProblem",
        (ElementwiseProblem,),
        {
            "__init__": lambda self: ElementwiseProblem.__init__(
                self,
                n_var=len(xl),
                n_obj=1,
                xl=xl,
                xu=xu,
                vtype=int,
            ),
            "_evaluate": _evaluate,
        },
    )
    return ProblemType()

problem = make_problem()

## Run GA and view results

In [ ]:
algorithm = GA(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=3),
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    get_termination("n_gen", 15),
    seed=42,
    verbose=True,
    save_history=True
)

## Show best solution

In [ ]:
best_x = np.asarray(res.X).astype(int)
best_plan = ou.decode_plan(best_x, UIV_CHOICES)

best_full = ou.build_full_input(patient_fixed, best_plan)
best_prob = ou.predict_mech_fail_prob(best_full, mech_fail_bundle)

print("Best mech_fail_prob:", best_prob)
best_plan

## Getting multiple solutions

Currently forcing ALIF to 1 for all solutions because that is all we can evaluate with training data

**Get top 10 solutions from last generation**

In [ ]:
pop = res.algorithm.pop
X_pop = pop.get("X")
F_pop = pop.get("F").flatten()

order = np.argsort(F_pop)

TOP_SLICE = 1500
TOP_N = 10

rows = []
for idx in order[:TOP_SLICE]:
    x = np.asarray(X_pop[idx]).astype(int)
    plan = ou.decode_plan(x, UIV_CHOICES)
    rows.append({
        **plan,
        "fitness": float(F_pop[idx]),
    })

df = pd.DataFrame(rows)

df_unique = df.drop_duplicates(
    subset=["UIV_implant", "num_fused_levels", "ALIF", "XLIF", "TLIF",
            "num_rods", "num_screws", "osteotomy"],
    keep="first",
).reset_index(drop=True)

df_unique.head(TOP_N)


In [ ]:
df["UIV_implant"].value_counts()


In [ ]:
solutions = s.get_diverse_solutions(
    res=res,
    top_n=5,
    top_per_gen=50,
    eps=0.1,
    bucket_cols=("UIV_implant", "XLIF", "TLIF"),
    n_per_bucket=1,
)

solutions
